In [ ]:
import os
import numpy as np
import pandas as pd
# jt -t monokai -f ubuntu -fs 14 -lineh 120 -ofs 10 -cellw 1500
pd.set_option('display.width',1000)

test_path = 'D:\\data\\Metro\\Metro_testA\\'
train_path = 'D:\\data\\Metro\\Metro_train\\'
out_path = 'D:\\data\\Metro\\midData\\'
f = os.listdir(train_path)

#exact trips

# for i in f:
#     print(i)
#     data = pd.read_csv(train_path + i)
#     data = data.sort_values(by=['userID', 'time']).reset_index(drop=True)
# #     print(data.loc[:20])
#     trips = pd.DataFrame(columns=['UID', 'PY', 'iT', 'iLID', 'iSID', 'iDID', 'oT', 'oLID', 'oSID', 'oDID'])
#     n = len(data)
#     for j in range(0, n):
#         if data.loc[j].status == 0 or j == n or data.loc[j+1].status == 1 or data.loc[j].userID != data.loc[j+1].userID: 
#             continue
#         trips.loc[len(trips)] = [data.loc[j].userID, data.loc[j].payType, data.loc[j].time, data.loc[j].lineID, 
#                                   data.loc[j].stationID, data.loc[j].deviceID, data.loc[j+1].time, data.loc[j+1].lineID,
#                                    data.loc[j+1].stationID, data.loc[j+1].deviceID]
#         print(trips.loc[len(trips)-1])
#     trips.to_csv(out_path + i.split('_')[1].split('.')[0] + '_trips.csv')

for i in f:
    print(i)
    data = pd.read_csv(train_path + i)
    data = data.drop_duplicates().sort_values(by=['userID', 'time']).reset_index(drop=True)
    data['idx'] = data.index
    indata = data[data.status == 1]
    outdata = data[data.status == 0]
    trips = pd.merge(indata, outdata, how='inner', on='userID')
    trips = trips[(trips.idx_x + 1 == trips.idx_y) & (trips.status_x == 1) & (trips.status_y == 0) & (trips.time_x < trips.time_y)].drop_duplicates()
    trips.drop(['status_x', 'status_y', 'idx_x', 'idx_y', 'payType_y'], axis=1, inplace=True)
    trips.columns = [ 'iT', 'iLID', 'iSID', 'iDID', 'UID', 'PY','oT', 'oLID', 'oSID', 'oDID']
    trips.to_csv(out_path + i.split('_')[1].split('.')[0] + '_trips.csv')

In [ ]:
import os
import numpy as np
import pandas as pd

in_path = 'D:\\data\\Metro\\midData\\trips\\'
out_path = 'D:\\data\\Metro\\midData\\flow_matrix\\'

f = os.listdir(in_path)
Map = pd.read_csv('D:\\data\\Metro\\Metro_roadMap.csv')
Map.shape
Map.drop('Unnamed: 0', axis=1, inplace=True)
Map = Map.values
for i in f:
    path = out_path + i.split('_')[0] + '\\'
    if not os.path.exists(path): os.makedirs(path)
    data = pd.read_csv(in_path + i)
    data['clock'] = data['iT'].apply(lambda x: int(x.split()[1].split(':')[0]))
    data['minute'] = data['iT'].apply(lambda x: int(x.split()[1].split(':')[1]))
    for clock in range(0, 24):
        for minute in range(0, 60, 10): 
            print('%d:%d' % (clock, minute))
            trips = data[(data['clock'] == clock) & (data['minute'] >= minute) & (data['minute'] < minute+10)];
            matrix = np.zeros([81,81])
            for iID in range(0, 81):
                for oID in range(0, 81):
                    if Map[iID][oID] == 0: continue
                    matrix[iID][oID] = trips[(trips.iSID == iID) & (trips.oSID == oID)].shape[0]
                
            matrix = pd.DataFrame(matrix)
            matrix.to_csv(path + str(clock) + '-' + str(minute) + '.csv')
                    

In [ ]:
import os
import numpy as np
import pandas as pd

# test_path = 'E:\\data\\Metro\\Metro_testA\\testA_record_2019-01-28.csv'
train_path = 'E:\\data\\Metro\\Metro_train\\'
file_list = os.listdir(train_path)

for file in file_list:
    data = pd.read_csv(train_path + file)
#     data.info()
    data = data.sort_values(by=['stationID', 'time'], ascending='true')

    data['clock'] = data['time'].apply(lambda x: int(x.split()[1].split(':')[0]))
    data['minute'] = data['time'].apply(lambda x: int(x.split()[1].split(':')[1]))

    result = pd.DataFrame(columns=['stationID', 'clock', 'minute', 'inNums', 'outNums'])
    for ID in range(0, 81):
        for clock in range(0, 24):
            for minute in range(0, 60, 10):
                status = data[(data['stationID'] == ID) & (data['clock'] == clock) & (data['minute'] >= minute) & (data['minute'] < minute+10)]['status'];
                result.loc[len(result)] = [ID, clock, minute, status.sum(), len(status)-status.sum()]
                print(result.loc[len(result)-1])

    result.to_csv('E:\\data\\Metro\\Metro_testA\\%s_flow.csv' % file, encoding='utf8')


In [ ]:
import os
import numpy as np
import pandas as pd
test_path = 'D:\\data\\Metro\\Metro_testA\\'
mid_path = 'D:\\data\\Metro\\midData\\day_flow\\'
f = os.listdir(mid_path)
# for i in f:
#     if i[0] != 'r': continue
#     oname = test_path + i
#     nname = test_path + i.split('.')[0].split('_')[1] + '_flow.csv'
#     os.rename(oname, nname)

except_list = ['01', '05', '06', '12', '13', '19', '20']
chs_list = ['22']
submit = pd.read_csv(test_path + "testA_submit_2019-01-29.csv")
cnt = 0
for i in f:
#     if i[0] != '2' or i.split('_')[0].split('-')[2] in except_list: continue
    if i[0] != '2' or i.split('_')[0].split('-')[2] not in chs_list: continue
    print(i)
    result = pd.read_csv(mid_path + i)
    if cnt != 0:
        submit['inNums'] += result['inNums']
        submit['outNums'] += result['outNums']
    else:
        submit['inNums'] = result['inNums']
        submit['outNums'] = result['outNums']
    cnt += 1

print(cnt)
submit['inNums'] /= cnt
submit['outNums'] /= cnt
submit.info()
submit.to_csv("testA_submit_2019-01-xx.csv", index=False, encoding='utf8')
submit